# What is a Workflow

One of the provisioned services made available to users of the EODH is the Workflow Runner (WR). More information about the WR is provided later on this page, but the fundamental question that requires answering is...

_<p style="text-align:center;"> In the context of the EODH, what is a workflow? </p>_

Using the terminology of the EODH, a workflow is a file written in CWL (the Common Workflow Language) that creates a data processing chain. However, this CWL file only provides the orchestration of a wider set of tools that are brought together as an entity known as an Earth Observation Application Package (EOAP).

# What is the Workflow Runner?

The WR is a required component of the Hub designed to interprate CWL files and ensure that the algorithms wrapped within them are sensibly scaled across the available infrastructure. In the case of the EODH, the infrastructure is a Kubernetes cluster running on an AWS backend, but this is not a hard-set requirement. 

The WR itself is a piece of software called ADES. If you want to understand the internal components of the WR then the ADES Design Document is avialable here: https://eoepca.github.io/proc-ades/master/. 

The use is that EOAPs can be transfered from platform to platform e.g. they could be developed on EODH and run on EOEPCA or the other way round. Platforms run by DLR and NASA also utilise EOAPs and procesisng algorithms can be shared between them.  

Good introduction here: https://carpentries-incubator.github.io/cwl-novice-tutorial/aio/index.html

# What is CWL?


cwltool is a great way to test workflows locally: https://cwltool.readthedocs.io/en/latest/cli.html

Good tutorial here: https://andrewjesaitis.com/posts/2017-02-06-cwl-tutorial/

Check your workflow: https://view.commonwl.org/

Examples - link to Lot 1 & 2
Other tools - link to Lot 2 tool


# How do I create a workflow?

The guidance around EOAPs is that handcrafting a package is possible but not recommended. These workflows can become very complex very quickly and it is recommended that some form of tooling is used to automate or semi-automate the generation of an EOAP. As part of the Pathfinder phase of the project, the EODH (via a 3rd party contractor, Oxidian) has created eoap-gen to help specialist technicians to create service workflows. 